In [7]:
import pandas as pd
import re
import json

#SHOWS MORE WHEN PRINTING DATA
pd.options.display.max_colwidth = 100

#READS IN THE TRANSACTION AND BALANCE JSON FILES FROM NORDIGEN AND DROPS THE TRANSACTION ID COLUMN SINCE IT HAS USELESS INFORMATION
df = pd.read_json("2a6e6c81-f4ab-42c3-81ea-b2dfa982ac59 (1).json")
df_transactions = pd.DataFrame(df['transactions']['booked'])
df_transactionAmount = pd.json_normalize(df_transactions['transactionAmount'])
df_transactions = pd.concat([df_transactions, df_transactionAmount], axis=1).drop('transactionAmount', axis = 1)
df_transactions.drop(columns=['transactionId'], inplace=True)

df = pd.read_json("balances-2a6e6c81-f4ab-42c3-81ea-b2dfa982ac59 (1).json")
#SINCE WE CHOSE THE BOOKED TRANSACTIONS, WE ONLY WANT TO SEE THE FORWARDAVAILABLE BALANCE (EXCLUDING UNCLEARED PAYMENTS)
df_balance = pd.DataFrame(df['balances'][1])

#CREATES A BALANCE COLUMN USING THE BALANCE JSON FILE, AND SUBTRACTING THE "AMOUNTS" IN THE TRANSACTION FILE
Current_Balance = []
Value = float(df_balance['balanceAmount'].loc['amount'])
for balance in df_transactions['amount']:
    Current_Balance.append(Value)
    Value -= float(balance)

df_transactions['balance'] = Current_Balance

In [8]:
finished = False
while not finished:
    counter = 0
    for index, description in enumerate(df_transactions['remittanceInformationUnstructured']):
        Tokens = re.findall('\s+', description)
        for token in Tokens:
            if len(token) > len('  '):
                df_transactions.loc[index,'remittanceInformationUnstructured'] = df_transactions.loc[index,'remittanceInformationUnstructured'].replace(' '*len(token),'  ')
                counter += 1
    if counter == 0:
        finished = True
        print("Finished!")
    else:
        print("Needs another go!")

Needs another go!
Needs another go!
Needs another go!
Finished!


In [9]:
df_transactions['remittanceInformationUnstructured'] = df_transactions['remittanceInformationUnstructured'].str.replace('  ',',').str.strip()

0                     8914 18NOV21 C,TFL TRAVEL CH,TFL.GOV.UK/CP GB
1                              8914 18NOV21 C,BEIGEL BAKE,LONDON GB
2                     8914 16NOV21 C,TFL TRAVEL CH,TFL.GOV.UK/CP GB
3                     8914 12NOV21 C,TFL TRAVEL CH,TFL.GOV.UK/CP GB
4     JACK,JACK,VIA ONLINE - PYMT FP 15/11/21 10,30002132470780000N
5                        8914 12NOV21,BOULEVARD,BRASSERIE,LONDON GB
6                                       8914 13NOV21,HRK,TALLINN EE
7                     8914 12NOV21 C,TFL TRAVEL CH,TFL.GOV.UK/CP GB
8                     8914 11NOV21 C,TFL TRAVEL CH,TFL.GOV.UK/CP GB
9                                8914 11NOV21 C,19 20 BAR,LONDON GB
10                               8914 11NOV21 C,19 20 BAR,LONDON GB
11                               8914 11NOV21 C,19 20 BAR,LONDON GB
12                    8914 09NOV21 C,TFL TRAVEL CH,TFL.GOV.UK/CP GB
13                          8914 08NOV21,TRADE OLD STREET,LONDON GB
14                         8914 07NOV21 C,HALA R

In [4]:
# Description_Cols = df_transactions["remittanceInformationUnstructured"].str.split(",", expand = True)
# print(Description_Cols)

                     0                                 1  \
0       8914 18NOV21 C                     TFL TRAVEL CH   
1       8914 18NOV21 C                       BEIGEL BAKE   
2       8914 16NOV21 C                     TFL TRAVEL CH   
3       8914 12NOV21 C                     TFL TRAVEL CH   
4                 JACK                              JACK   
5         8914 12NOV21                         BOULEVARD   
6         8914 13NOV21                               HRK   
7       8914 12NOV21 C                     TFL TRAVEL CH   
8       8914 11NOV21 C                     TFL TRAVEL CH   
9       8914 11NOV21 C                         19 20 BAR   
10      8914 11NOV21 C                         19 20 BAR   
11      8914 11NOV21 C                         19 20 BAR   
12      8914 09NOV21 C                     TFL TRAVEL CH   
13        8914 08NOV21                  TRADE OLD STREET   
14      8914 07NOV21 C                   HALA RESTAURANT   
15      8914 08NOV21 C                  

In [17]:
df = pd.read_csv('Transaction Explanations.txt',sep = ';')
df_transactions.rename(columns = {'proprietaryBankTransactionCode':'Transaction Code'}, inplace = True)
df_transactions = df_transactions.merge(df, on ='Transaction Code', how='left')

   Transaction Code                  Description  \
0               103                MT103 Payment   
1               ACI  Interest on Account Balance   
2               ADV              Separate Advice   
3               AMD           Amendments History   
4               ATM              Cash Withdrawal   
..              ...                          ...   
71              TSU            Telephone Banking   
72              U/D          Unpaid Direct Debit   
73              UTF              Urgent Transfer   
74              WSF        Foreign Exchange Deal   
75              WSM            Money Market Deal   

                                                                                            Explanation  
0   Standardised SWIFT payment message, used specifically for cross border/international wire transf...  
1   Amount of money a bank, or financial institution pays a depositor for holding their money with t...  
2                                                        

In [23]:
print(df_transactions)

   bookingDate Transaction Code  \
0   2021-11-19              POS   
1   2021-11-19              POS   
2   2021-11-17              POS   
3   2021-11-15              POS   
4   2021-11-15              DPC   
5   2021-11-15              POS   
6   2021-11-15              POS   
7   2021-11-15              POS   
8   2021-11-12              POS   
9   2021-11-12              POS   
10  2021-11-12              POS   
11  2021-11-12              POS   
12  2021-11-10              POS   
13  2021-11-09              POS   
14  2021-11-09              POS   
15  2021-11-09              POS   
16  2021-11-08              POS   
17  2021-11-03              POS   
18  2021-11-02              POS   
19  2021-11-02              POS   
20  2021-11-01              POS   
21  2021-11-01              POS   
22  2021-10-29              BAC   
23  2021-10-28              POS   
24  2021-10-27              POS   
25  2021-10-27              POS   
26  2021-10-27              POS   
27  2021-10-25      